In [14]:
from a3z_scraper import get_a3z_game_data
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import urllib.request
import json
from datetime import datetime, timedelta

In [2]:
a3z_pbp = get_a3z_game_data()

In [3]:
a3z_pbp.head()

,Period,Time,Strength,Team,Shooter,Shot Type,A1,A2,A3,A1 Zone,...,Lane,Dump recovered?,Chance?,Retrieval,Result,Pressure,Exit,Result.1,Game ID,Seconds
0,1.0,18:01,5v5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,20489.0,119
1,1.0,17:58,5v5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5NYR,BOT,NaN,NaN,NaN,20489.0,122
2,1.0,17:41,5v5,PHI,21.0,w,62.0,20.0,NaN,oelc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20489.0,139
3,1.0,17:40,5v5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,72NYR,EXC,20PHI,13NYR,FEX,20489.0,140
4,1.0,17:34,5v5,PHI,62.0,w,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20489.0,146


In [4]:
# 0:3 in all tables as the recombination
# 3:20 is First group
shots = a3z_pbp.iloc[:, 0:20]
shots = shots.dropna(subset=list(shots.columns[3:]), how='all')
# 20:27 is U:AA color starts at 101
zone_entries = a3z_pbp.iloc[:, np.r_[0:3, 20:27]]
zone_entries = zone_entries.dropna(subset=list(zone_entries.columns[3:]), how='all')
# 27:32 is AB:AG color starts at 223
zone_exits = a3z_pbp.iloc[:, np.r_[0:3, 27:32]]
zone_exits = zone_exits.dropna(subset=list(zone_exits.columns[3:]), how='all')

In [ ]:
zone_entries.head()

In [5]:
# ZONE ENTRY TABLE MANIPULATIONS
import re
zone_entries = a3z_pbp.iloc[:, np.r_[0:3, 20:27]]
zone_entries = zone_entries.dropna(subset=list(zone_entries.columns[3:]), how='all')
zone_entries = zone_entries.drop(zone_entries[zone_entries['Entry Type'] == 'X'].index)
zone_entries['Team'] = zone_entries["Entry By"].apply(lambda x: "".join(list(filter(str.isalpha, x))))
zone_entries['Player'] = zone_entries["Entry By"].apply(lambda x: int("".join(list(filter(str.isdigit, x)))))
zone_entries['dTeam'] = zone_entries["Defended by"].apply(lambda x: "".join(list(filter(str.isalpha, x))))
zone_entries['dPlayer'] = zone_entries["Defended by"].apply(lambda x: "".join(list(filter(str.isdigit, x))))
zone_entries['Recovery'] = zone_entries["Dump recovered?"].apply(lambda x: "".join(list(filter(str.isdigit, x))))
zone_entries['Event'] = 'Zone Entry'

entry_type = {
  'D':'DUMP',
  'C':'CARRY',
  'F':'FAILED_ENTRY',
}
zone_entries['Type'] = zone_entries[['Entry Type', 'Pass?']].apply(lambda x: entry_type[x['Entry Type']] if not (entry_type[x['Entry Type']] and x['Pass?'] == 'Y') else 'CARRIED_WITH_PASS', axis=1)
zone_entries.rename(columns={'Chance?': 'Chance'}, inplace=True)
# CARRIED WITH PASS (where pass is Y)
zone_entries_final = zone_entries[['Period', 'Time', 'Strength', 'Event', 'Type', 'Team', 'Player', 'dTeam', 'dPlayer', 'Lane', 'Recovery', 'Chance']]
zone_entries_final.head(10)

,Period,Time,Strength,Event,Type,Team,Player,dTeam,dPlayer,Lane,Recovery,Chance
7,1.0,17:15,5v5,Zone Entry,DUMP,NYR,91,N,,R,,N
10,1.0,16:34,5v5,Zone Entry,DUMP,PHI,49,N,,C,74,Y
13,1.0,16:05,5v5,Zone Entry,DUMP,NYR,8,N,,R,,N
15,1.0,15:57,5v5,Zone Entry,CARRIED_WITH_PASS,PHI,11,NYR,79,L,,N
18,1.0,15:48,5v5,Zone Entry,CARRY,NYR,10,PHI,77,L,,N
21,1.0,15:21,5v5,Zone Entry,DUMP,NYR,20,N,,R,,N
23,1.0,15:08,5v5,Zone Entry,DUMP,PHI,20,N,,L,,N
25,1.0,14:41,5v5,Zone Entry,CARRIED_WITH_PASS,NYR,72,PHI,9,R,,Y
28,1.0,14:30,5v5,Zone Entry,DUMP,PHI,74,NYR,55,R,,N
30,1.0,14:08,5v5,Zone Entry,DUMP,PHI,6,N,,L,,N


In [6]:
# ZONE EXIT TABLE MANIPULATIONS
zone_exits = a3z_pbp.iloc[:, np.r_[0:3, 27:32]]
zone_exits = zone_exits.dropna(subset=list(zone_exits.columns[3:]), how='all')
zone_exits.fillna('N',inplace=True)
zone_exits.head()
zone_exits['Team'] = zone_exits["Retrieval"].apply(lambda x: "".join(list(filter(str.isalpha, x))))
zone_exits['Player'] = zone_exits[["Retrieval","Exit"]].apply(lambda x: "".join(list(filter(str.isdigit, x['Retrieval']))) if x['Exit'] == 'N' else "".join(list(filter(str.isdigit, x['Exit']))),axis=1)
zone_exits['Retrieval'] = zone_exits[["Retrieval","Exit"]].apply(lambda x: "".join(list(filter(str.isdigit, x['Retrieval']))) if x['Exit'] != 'N' else 'N',axis=1)
zone_exits['dTeam'] = zone_exits["Pressure"].apply(lambda x: "".join(list(filter(str.isalpha, x))))
zone_exits['dPlayer'] = zone_exits["Pressure"].apply(lambda x: "".join(list(filter(str.isdigit, x))))
zone_exits['Event'] = 'Zone Exit'
# shots[['SOG?', 'G?']].apply(lambda x: 'Goal' if x['G?'] == 'y' else (('Shot' if x['SOG?'] == 'y' else 'Missed Shot')), axis=1)
exit_types = {
  'BOT': 'BOTCHED_RETRIEVAL',
  'FEX': 'FAILED_EXIT',
  'PEX': 'PASSED',
  'MEX': 'MISSED_PASS',
  'CEX': 'CARRIED',
  'CLE': 'CLEAR',
  'I': 'ICING',
}
zone_exits['Type'] = zone_exits[['Result','Result.1']].apply(lambda x: exit_types[x['Result']] if x['Result'] != 'EXC' else (exit_types[x['Result.1']] if x['Result.1'] != 'N' else 'BOTCHED_RETRIEVAL'), axis=1)
# Switch exit and Retrieval Number
# 'Period', 'Time', 'Strength', 'Event', 'Type', 'Team', 'Number', 'dTeam', 'dNumber', 'Recovery'
zone_exits_final = zone_exits[['Period', 'Time', 'Strength', 'Event', 'Type', 'Team', 'Player', 'dTeam', 'dPlayer', 'Retrieval']]
zone_exits_final.head(20)

,Period,Time,Strength,Event,Type,Team,Player,dTeam,dPlayer,Retrieval
1,1.0,17:58,5v5,Zone Exit,BOTCHED_RETRIEVAL,NYR,5,N,,N
3,1.0,17:40,5v5,Zone Exit,FAILED_EXIT,NYR,13,PHI,20,72
5,1.0,17:30,5v5,Zone Exit,MISSED_PASS,NYR,74,N,,23
6,1.0,17:20,5v5,Zone Exit,PASSED,NYR,23,N,,N
8,1.0,17:12,5v5,Zone Exit,BOTCHED_RETRIEVAL,PHI,55,NYR,76,N
12,1.0,16:13,5v5,Zone Exit,MISSED_PASS,PHI,61,N,,N
14,1.0,16:02,5v5,Zone Exit,PASSED,PHI,79,N,,N
17,1.0,15:51,5v5,Zone Exit,PASSED,NYR,79,N,,N
20,1.0,15:31,5v5,Zone Exit,MISSED_PASS,PHI,77,NYR,16,445
22,1.0,15:15,5v5,Zone Exit,CLEAR,PHI,62,N,,77


In [ ]:
shots.head()

In [12]:
# SHOT TABLE SPLIT FOR PASSES MANIPULATIONS
pass_types = {
    'p': 'low_to_high',
    's': 'strech',
    'f': 'faceoff',
    'rr':'slot',
    'el': 'below_goal_line',
}

zones = {
    'o': 'offensive',
    'd': 'defensive',
    'n': 'neutral',
    'oz':'offensive',
    'dz': 'defensive',
    'nz':'neutral',
}

lanes = {
    'l': 'left',
    'r': 'right',
    'c': 'center',
    'L':'left',
    'R': 'right',
    'C': 'center',
}

shot_types = {
  'a': 'wrap_around',
  'b': 'backhander',
  's': 'slapshot',
  'w': 'wrist_shot',
  'o': 'one_timer',
  'r': 'rebound',
  't': 'tip'
}
play_types = {
    'f': 'forecheck',
    'r': 'rush',
    'c': 'cycle',
}


shots = a3z_pbp.iloc[:, 0:20]
shots = shots.dropna(subset=list(shots.columns[3:]), how='all')
shots[['Shooter','A1','A2','A3']] = (shots[['Shooter','A1','A2','A3']].fillna(-1)).astype('int').astype('str').replace('-1','N')
shots['Event'] = shots[['SOG?', 'G?']].apply(lambda x: 'Goal' if x['G?'] == 'y' else (('Shot' if x['SOG?'] == 'y' else 'Missed Shot')), axis=1)
# Period Time Strength Event Type Team Number primaryAssist secondaryAssist tertiaryAssist Chance playType oddMan screened origin 
#             primaryZone primaryLane primaryPassType secondaryZone secondaryLane secondaryPassType tertiaryZone tertiaryLane tertiaryPassType
shots['primaryZone'] = shots['A1 Zone'].apply(lambda x: zones[str(x)[0]] if (str(x) != 'nan') and (str(x)[0] in zones) else '')
shots['primaryLane'] = shots['A1 Zone'].apply(lambda x: lanes[str(x)[-1]] if str(x)[-1] in lanes else '')
shots['primaryPassType'] = shots['A1 Zone'].apply(lambda x: pass_types[str(x)[1:-1]] if str(x)[1:-1] in pass_types else '')
shots['secondaryZone'] = shots['A2 Zone'].apply(lambda x: zones[str(x)[0]] if (str(x) != 'nan') and (str(x)[0] in zones) else '')
shots['secondaryLane'] = shots['A2 Zone'].apply(lambda x: lanes[str(x)[-1]] if str(x)[-1] in lanes else '')
shots['secondaryPassType'] = shots['A2 Zone'].apply(lambda x: pass_types[str(x)[1:-1]] if (str(x)[1:-1]) in pass_types else '')
shots['tertiaryZone'] = shots['A3 Zone'].apply(lambda x: zones[str(x)[0]] if (str(x) != 'nan') and (str(x)[0] in zones) else '')
shots['tertiaryLane'] = shots['A3 Zone'].apply(lambda x: lanes[str(x)[-1]] if str(x)[-1] in lanes else '')
shots['tertiaryPassType'] = shots['A3 Zone'].apply(lambda x: pass_types[str(x)[1:-1]] if str(x)[1:-1] in pass_types else '')
shots.rename(columns={'Shooter': 'Player','A1': 'primaryAssist', 'A2': 'secondaryAssist', 'A3': 'tertiaryAssist', 'SC?': 'Chance', 'Oddman?': 'Oddman'}, inplace=True)
shots['Type'] = shots['Shot Type'].apply(lambda x: shot_types[x.lower()])
shots['Origin'] = shots['Origin'].apply(lambda x: zones[x] if x in zones else x)
shots['playType'] = shots['Rush?'].apply(lambda x: play_types[x] if x in play_types else x)
shots_final = shots[['Period', 'Time', 'Strength', 'Event', 'Type', 
                     'Team', 'Player', 'primaryAssist','secondaryAssist','tertiaryAssist', 'Chance', 'playType', 'Oddman','primaryZone',
                     'secondaryZone','tertiaryZone', 'primaryLane','secondaryLane','tertiaryLane',
                     'primaryPassType','secondaryPassType','tertiaryPassType']]
shots_final.head()

,Period,Time,Strength,Event,Type,Team,Player,primaryAssist,secondaryAssist,tertiaryAssist,...,Oddman,primaryZone,secondaryZone,tertiaryZone,primaryLane,secondaryLane,tertiaryLane,primaryPassType,secondaryPassType,tertiaryPassType
2,1.0,17:41,5v5,Missed Shot,wrist_shot,PHI,21,62,20,N,...,NaN,offensive,offensive,,center,left,,below_goal_line,,
4,1.0,17:34,5v5,Shot,wrist_shot,PHI,62,N,N,N,...,NaN,,,,,,,,,
9,1.0,17:04,5v5,Missed Shot,wrist_shot,NYR,55,23,91,N,...,NaN,offensive,offensive,,right,right,,,low_to_high,
11,1.0,16:26,5v5,Shot,wrist_shot,PHI,25,49,N,N,...,NaN,offensive,,,right,,,below_goal_line,,
16,1.0,15:54,5v5,Missed Shot,wrist_shot,PHI,49,11,86,N,...,NaN,offensive,defensive,,left,left,,,,


In [54]:
a3z_final = pd.concat([zone_entries_final, zone_exits_final, shots_final]).sort_index()
a3z_final = pd.merge(a3z_final,a3z_pbp[['Period', 'Time', 'Seconds']],on=['Period','Time'], how='left')
a3z_final.head()

,Period,Time,Strength,Event,Type,Team,Player,dTeam,dPlayer,Lane,...,primaryZone,secondaryZone,tertiaryZone,primaryLane,secondaryLane,tertiaryLane,primaryPassType,secondaryPassType,tertiaryPassType,Seconds
0,1.0,17:58,5v5,Zone Exit,BOTCHED_RETRIEVAL,NYR,5,N,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122
1,1.0,17:41,5v5,Missed Shot,wrist_shot,PHI,21,NaN,NaN,NaN,...,offensive,offensive,,center,left,,below_goal_line,,,139
2,1.0,17:40,5v5,Zone Exit,FAILED_EXIT,NYR,13,PHI,20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140
3,1.0,17:34,5v5,Shot,wrist_shot,PHI,62,NaN,NaN,NaN,...,,,,,,,,,,146
4,1.0,17:30,5v5,Zone Exit,MISSED_PASS,NYR,74,N,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150


In [37]:

def update_player_ids_from_json(teamdata):
    """
    Creates a data frame of player data from current game's json[liveData][boxscore] to update player ids.
    This method reads player ids, names, handedness, team, position, and number, and full joins to player ids.
    If there are any changes to player ids, the dataframe gets written to disk again.
    Parameters
    -----------
    teamdata : dict
        A json dict that is the result of api_page['liveData']['boxscore']['teams']
    """

    awayplayers = teamdata["away"]["players"]
    homeplayers = teamdata["home"]["players"]

    numplayers = len(awayplayers) + len(homeplayers)
    ids = ["" for i in range(numplayers)]
    names = ["" for i in range(numplayers)]
    teams = ["" for i in range(numplayers)]
    positions = ["" for i in range(numplayers)]
    nums = [-1 for i in range(numplayers)]
    handedness = ["" for i in range(numplayers)]

    for i, (pid, pdata) in enumerate(awayplayers.items()):
        idnum = pid[2:]
        name = pdata["person"]["fullName"]
        try:
            hand = pdata["person"]["shootsCatches"]
        except KeyError:
            hand = "N/A"
        try:
            num = pdata["jerseyNumber"]
            if num == "":
                raise KeyError
            num = int(num)
        except KeyError:
            num = -1
        pos = pdata["position"]["code"]

        ids[i] = idnum
        names[i] = name
        teams[i] = teamdata['away']["team"]["triCode"]
        positions[i] = pos
        nums[i] = num
        handedness[i] = hand

    for i, (pid, pdata) in enumerate(homeplayers.items()):
        idnum = pid[2:]
        name = pdata["person"]["fullName"]
        try:
            hand = pdata["person"]["shootsCatches"]
        except KeyError:
            hand = "N/A"
        try:
            num = pdata["jerseyNumber"]
            if num == "":
                raise KeyError
            num = int(num)
        except KeyError:
            num = -1
        pos = pdata["position"]["code"]

        ids[i + len(awayplayers)] = idnum
        names[i + len(awayplayers)] = name
        teams[i + len(awayplayers)] = teamdata['home']["team"]["triCode"]
        positions[i + len(awayplayers)] = pos
        nums[i + len(awayplayers)] = num
        handedness[i + len(awayplayers)] = hand
    gamedf = pd.DataFrame(
        {
            "ID": ids,
            "Name": names,
            "Team": teams,
            "Pos": positions,
            "#": nums,
            "Hand": handedness,
        }
    )
    return gamedf


def update_team_ids_from_json(teamdata):
    import urllib.request
    import json
    import pandas as pd

    hid = teamdata["home"]["team"]["id"]
    url = "https://statsapi.web.nhl.com{0:s}".format(
        teamdata["home"]["team"]["link"]
    )
    with urllib.request.urlopen(url) as reader:
        page = reader.read()
    teaminfo = json.loads(page.decode("latin-1"))
    hid = teaminfo["teams"][0]["id"]
    habbrev = teaminfo["teams"][0]["abbreviation"]
    hname = teaminfo["teams"][0]["name"]

    team_ids = pd.DataFrame({"ID": [hid], "Abbreviation": [habbrev], "Name": [hname]})

    rid = teamdata["away"]["team"]["id"]
    if rid not in team_ids.ID.values:
        url = "https://statsapi.web.nhl.com{0:s}".format(
            teamdata["away"]["team"]["link"]
        )
        with urllib.request.urlopen(url) as reader:
            page = reader.read()
        teaminfo = json.loads(page.decode("latin-1"))
        rid = teaminfo["teams"][0]["id"]
        rabbrev = teaminfo["teams"][0]["abbreviation"]
        rname = teaminfo["teams"][0]["name"]

        df = pd.DataFrame({"ID": [rid], "Abbreviation": [rabbrev], "Name": [rname]})
        team_ids = pd.concat([team_ids, df])
    return team_ids

def read_events_from_json(pbp):
    """
    Returns the NHL API url to scrape.
    Parameters
    -----------
    season : int
        The season of the game. 2007-08 would be 2007.
    game : int
        The game id. This can range from 20001 to 21230 for regular season, and 30111 to 30417 for playoffs.
        The preseason, all-star game, Olympics, and World Cup also have game IDs that can be provided.
    Returns
    --------
    pandas df
        Dataframe of the game's play by play data
    """
    index = list(range(len(pbp)))
    period = [-1 for i in range(len(pbp))]
    time = ["0:00" for i in range(len(pbp))]
    dateTime = [-1 for i in range(len(pbp))]
    event = ["NA" for i in range(len(pbp))]

    team = ["" for i in range(len(pbp))]
    p1 = [-1 for i in range(len(pbp))]
    p1role = ["" for i in range(len(pbp))]
    p2 = [-1 for i in range(len(pbp))]
    p2role = ["" for i in range(len(pbp))]
    xy = [(np.NaN, np.NaN) for i in range(len(pbp))]
    note = ["" for i in range(len(pbp))]

    for i in range(len(pbp)):
        period[i] = int(pbp[i]["about"]["period"])
        time[i] = pbp[i]["about"]["periodTimeRemaining"]

        dateTime[i] = utc_to_est(  # type: ignore
            datetime.fromisoformat(pbp[i]["about"]["dateTime"][:-1])
        )
        event[i] = pbp[i]["result"]["event"]

        try:
            xy[i] = (
                float(pbp[i]["coordinates"]["x"]),
                float(pbp[i]["coordinates"]["y"]),
            )
        except KeyError:
            pass
        try:
            team[i] = pbp[i]["team"]["triCode"]
        except KeyError:
            pass
        try:
            p1[i] = pbp[i]["players"][0]["player"]["id"]
            p1role[i] = pbp[i]["players"][0]["playerType"]
        except KeyError:
            pass
        try:
            p2[i] = pbp[i]["players"][1]["player"]["id"]
            p2role[i] = pbp[i]["players"][1]["playerType"]
        except KeyError:
            pass
        except IndexError:  # e.g. on a give or take
            pass

        try:
            note[i] = pbp[i]["result"]["description"]
        except KeyError:
            pass

        # print(period[i], time[i], event[i], xy[i], team[i], p1[i], p1role[i], p2[i], p2role[i])

    pbpdf = pd.DataFrame(
        {
            "Index": index,
            "Period": period,
            "Time": time,
            "DateTime": dateTime,
            "Event": event,
            "Team": team,
            "Actor": p1,
            "ActorRole": p1role,
            "Recipient": p2,
            "RecipientRole": p2role,
            "XY": xy,
            "Note": note,
        }
    )
    return pbpdf
# def update_quick_gamelog_from_json(data):
#     """
#     Creates a data frame of basic game data from current game's json to update global BASIC_GAMELOG.
#     This method reads the season, game, date and time, venue, and team names, coaches, anc scores, joining to
#     BASIC_GAMELOG.
#     If there are any changes to BASIC_GAMELOG, the dataframe gets written to disk again.
#     Parameters
#     -----------
#     data : dict
#         The full json dict from the api_page
#     """
#     season = int(str(data["gameData"]["game"]["pk"])[:4])
#     game = int(str(data["gameData"]["game"]["pk"])[4:])
#     datetime = data["gameData"]["datetime"]["dateTime"]
#     try:
#         venue = data["gameData"]["venue"]["name"]
#     except KeyError:
#         venue = "N/A"
#     hname = team_ids.query("ID == " + str(data["gameData"]["teams"]["home"]["id"]))
#     hname = hname["Abbreviation"].iloc[0]
#     rname = team_ids.query("ID == " + str(data["gameData"]["teams"]["away"]["id"]))
#     rname = rname["Abbreviation"].iloc[0]
#     try:
#         hcoach = data["liveData"]["boxscore"]["teams"]["home"]["coaches"][0]["person"][
#             "fullName"
#         ]
#     except IndexError:
#         hcoach = "N/A"
#     try:
#         rcoach = data["liveData"]["boxscore"]["teams"]["away"]["coaches"][0]["person"][
#             "fullName"
#         ]
#     except IndexError:
#         rcoach = "N/A"
#     hscore = data["liveData"]["boxscore"]["teams"]["home"]["teamStats"][
#         "teamSkaterStats"
#     ]["goals"]
#     rscore = data["liveData"]["boxscore"]["teams"]["away"]["teamStats"][
#         "teamSkaterStats"
#     ]["goals"]

#     import pandas as pd

#     gamedf = pd.DataFrame(
#         {
#             "Season": [season],
#             "Game": [game],
#             "Datetime": [datetime],
#             "Venue": [venue],
#             "Home": [hname],
#             "HomeCoach": [hcoach],
#             "HomeScore": [hscore],
#             "Away": [rname],
#             "AwayCoach": [rcoach],
#             "AwayScore": [rscore],
#         }
#     ) 
#     basic_gamelog = pd.concat([basic_gamelog, gamedf]).drop_duplicates()

In [41]:
def utc_to_est(utc_dt):
    return utc_dt - timedelta(hours=5)
def get_nhl_game_events(gameId):
    with urllib.request.urlopen(
        f"https://statsapi.web.nhl.com/api/v1/game/{gameId}/feed/live"
    ) as reader:
        page = reader.read()

    data = json.loads(page.decode("latin-1"))
    start_time = utc_to_est(
        datetime.fromisoformat(data["gameData"]["datetime"]["dateTime"][:-1])
    )
    teamdata = data['liveData']['boxscore']['teams']
    events = read_events_from_json(data["liveData"]["plays"]["allPlays"])
    events.drop(events[events["Team"] == ""].index, inplace=True)
    events["datetime_shifted"] = events["DateTime"].shift(1)
    events["datetime_shifted"].iloc[0] = start_time
    events["time_since_last_event"] = (events.DateTime - events.datetime_shifted).apply(
        lambda x: x.total_seconds()
    )

    def convert_game_time(period, time):
        time_list = time.split(":")
        clock = (int(time_list[0]) * 60) + (int(time_list[1]))
        return int(period * 1200 - clock)

    events["Time"] = events["Time"].astype(str)
    events["Seconds"] = events.apply(
        lambda x: convert_game_time(x["Period"], x["Time"]), axis=1
    )
    players = update_player_ids_from_json(teamdata)
    teams = update_team_ids_from_json(teamdata)
    return (players, teams, events)

In [42]:
(players, teams, events) = get_nhl_game_events(2022020489)

/var/folders/ld/c6zftrds33ngqn6mtmn2nlcm0000gn/T/ipykernel_99275/747977427.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events["datetime_shifted"].iloc[0] = start_time


In [64]:
def merge_a3z_and_nhl():
    a3z_pbp = a3z_final
    (_,_,nhl_pbp_data) = get_nhl_game_events(2022020489)
    # f"{video_url}?fromTime={buffer}"
    merged_df = (
        a3z_pbp.merge(nhl_pbp_data, on=["Seconds", "Team"], how="outer")
        .sort_values("Seconds")
        .reset_index()
    )
    return merged_df[
        [
            "Team",
            "Event_x",
            "Event_y",
            "Seconds",
            "Period_x",
            "Time_x",
            "Period_y",
            "Time_y",
            "DateTime",
            "time_since_last_event",
        ]
    ]
df = merge_a3z_and_nhl()
df.drop(df[df["Team"] == ""].index, inplace=True)
df.head()

/var/folders/ld/c6zftrds33ngqn6mtmn2nlcm0000gn/T/ipykernel_99275/747977427.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events["datetime_shifted"].iloc[0] = start_time


,Team,Event_x,Event_y,Seconds,Period_x,Time_x,Period_y,Time_y,DateTime,time_since_last_event
0,NYR,NaN,Faceoff,0,NaN,NaN,1.0,20:00,2022-12-17 19:08:25,505.0
1,NYR,Zone Exit,NaN,122,1.0,17:58,NaN,NaN,NaT,NaN
2,PHI,Missed Shot,Blocked Shot,139,1.0,17:41,1.0,17:41,2022-12-17 19:10:45,140.0
3,NYR,Zone Exit,NaN,140,1.0,17:40,NaN,NaN,NaT,NaN
4,PHI,NaN,Shot,145,NaN,NaN,1.0,17:35,2022-12-17 19:10:51,6.0


In [70]:
import requests
import sys
import re
import pandas as pd
def convertTime(timeString,period):
    timeString=timeString.strip()
    time=60*int(timeString.split(":")[0])+int(timeString.split(":")[1])
    return time + 20*60*max(0,period-1)
# implement default args
season = sys.argv[1]
gameId = sys.argv[2]

awayPage = requests.get(
    "http://www.nhl.com/scores/htmlreports/20222023/TV020489.HTM"
)
# awayPage = requests.get('http://www.nhl.com/scores/htmlreports/20142015/TV030227.HTM')
homePage = requests.get(
    "http://www.nhl.com/scores/htmlreports/20222023/TH020489.HTM"
)
# homePage = requests.get('http://www.nhl.com/scores/htmlreports/20142015/TH030227.HTM')
def parseTOI(text):
    playersText=text.split("playerHeading")

    # Game meta-data
    top=playersText[1]
    gameNumber=re.search(r':bold">Game (\d+)',top).group(1)
    gameDate=re.search(r'bold">(\w+, \w+ \d+, \d+)</td>',top).group(1)
    teamName=re.search(r'class="teamHeading \+ border" align="center">([\w \.-]+)</td',top).group(1)

    # Process shift-by-shift data
    dfList=[]
    for playerText in playersText[2:]:
        shifts=playerText.split("</tr>")
        nameNum=re.search('colspan="8">(\d+) ([\w \.\'-]+), ([\w \.\'-]+)',shifts[0])
        playerNumber, lastName, firstName=nameNum.groups()
        # Text to find name/number
        shifts=playerText.split("<tr")
        shiftNumber=[]
        shiftPeriod=[]
        shiftStart=[]
        shiftEnd=[]
        for shiftText in shifts[2:]:
            shiftData=re.findall(r'border">([\w :]+)',shiftText)
            # Break when hitting summaries
            if not shiftData or shiftData[0]=="Per": break

            # Convert to Numeric
            period=shiftData[1]
            if period=="OT": period=4
            else: period=int(period)
            shiftNumber.append(int(shiftData[0]))
            shiftPeriod.append(period)

            # Convert to TimeData
            shiftStart.append(convertTime(shiftData[2],period))
            shiftEnd.append(convertTime(shiftData[3],period))
        
        playerDf=pd.DataFrame({
          'playerNumber': playerNumber,
          'team': teamName,
          'shiftNumber': shiftNumber,
          'shiftPeriod': shiftPeriod,
          'shiftStart': shiftStart,
          'shiftEnd': shiftEnd,
          'gameNumber': gameNumber,
          'gameDate':gameDate})
        dfList.append(playerDf)
    return pd.concat(dfList)
parseTOI(awayPage.text).head()
parseTOI(homePage.text).head()

,playerNumber,team,shiftNumber,shiftPeriod,shiftStart,shiftEnd,gameNumber,gameDate
0,6,PHILADELPHIA FLYERS,1,1,45,88,0489,"Saturday, December 17, 2022"
1,6,PHILADELPHIA FLYERS,2,1,202,245,0489,"Saturday, December 17, 2022"
2,6,PHILADELPHIA FLYERS,3,1,328,369,0489,"Saturday, December 17, 2022"
3,6,PHILADELPHIA FLYERS,4,1,431,459,0489,"Saturday, December 17, 2022"
4,6,PHILADELPHIA FLYERS,5,1,580,646,0489,"Saturday, December 17, 2022"
